In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# sk learn imports for the ML model
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

## MLFLow Import

In [17]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location=('/workspaces/MLOps-Model-Development-to-Production-deployment/Experiment '
 'Tracking using MLFlow/mlruns/1'), creation_time=1749424860439, experiment_id='1', last_update_time=1749424860439, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
df = pd.read_parquet('/workspaces/MLOps-Model-Development-to-Production-deployment/Data/green_tripdata_2025-01.parquet')
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-01-01 00:03:01,2025-01-01 00:17:12,N,1.0,75,235,1.0,5.93,24.70,...,0.5,6.80,0.00,NaN,1.0,34.00,1.0,1.0,0.00,0.0
1,2,2025-01-01 00:19:59,2025-01-01 00:25:52,N,1.0,166,75,1.0,1.32,8.60,...,0.5,0.00,0.00,NaN,1.0,11.10,2.0,1.0,0.00,0.0
2,2,2025-01-01 00:05:29,2025-01-01 00:07:21,N,5.0,171,73,1.0,0.41,25.55,...,0.0,0.00,0.00,NaN,1.0,26.55,2.0,2.0,0.00,0.0
3,2,2025-01-01 00:52:24,2025-01-01 01:07:52,N,1.0,74,223,1.0,4.12,21.20,...,0.5,6.13,6.94,NaN,1.0,36.77,1.0,1.0,0.00,0.0
4,2,2025-01-01 00:25:05,2025-01-01 01:01:10,N,1.0,66,158,1.0,4.71,33.80,...,0.5,7.81,0.00,NaN,1.0,46.86,1.0,1.0,2.75,0.0


## Preprocessing of data

In [4]:
df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [5]:
train_dicts = df[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred)

44.94737843983856

In [6]:
def preprocess_df(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train = preprocess_df('/workspaces/MLOps-Model-Development-to-Production-deployment/Data/green_tripdata_2025-01.parquet')
df_val = preprocess_df('/workspaces/MLOps-Model-Development-to-Production-deployment/Data/green_tripdata_2025-02.parquet')

print(df_train.shape, df_val.shape)

(46307, 22) (44218, 22)


In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

37.19536000471952

In [11]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [12]:
## define a new run and put in the ml flow run

with mlflow.start_run():

    mlflow.set_tag("developer", "keyush")

    ## we can also log a different params
    mlflow.log_param("train-dataPath", "/workspaces/MLOps-Model-Development-to-Production-deployment/Data/green_tripdata_2025-01.parquet")
    mlflow.log_param("val-dataPath", "/workspaces/MLOps-Model-Development-to-Production-deployment/Data/green_tripdata_2025-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("mse", mse)

In [13]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [14]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

### fmin()
This is the main minimization function.
You pass a loss function (e.g., RMSE), a search space, and an algorithm.
It returns the best set of hyperparameters.

#### tpe (Tree-structured Parzen Estimator)
A Bayesian optimization algorithm.
Learns from previous trials to decide what parameters to try next (smarter than random search).

#### hp

Used to define the search space of hyperparameters.
Examples:

hp.uniform('learning_rate', 0.01, 0.3)
hp.choice('max_depth', [3, 5, 7, 10])

#### scope (from hyperopt.pyll)
Used to apply transformations to sampled hyperparameters.

Example:

scope.int(hp.quniform('max_depth', 3, 10, 1))
This means: sample integers from 3 to 10 (step 1).


In [18]:
def objective_eval(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        xg = xgb.train(
            params,
            dtrain = train,
            num_boost_round = 1000, ## there will be 1000 iterations
            evals=[(valid, 'validation')],
            early_stopping_rounds=50 ## if there is no improvement then the model will stop. That is why you will less than 1000 rounds below in 
            ## each of the runs which is like an epoch where a new model is instantiated and run!
        ) 
        y_pred = xg.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {"loss": rmse, "status": STATUS_OK}

In [19]:
## hp helps us to modify the parameters, 

search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), ## here we find a value between e^(-3) whch is 0.05 and exp(0) which is 1
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear', ## this is a reg problem and the scope of the target is continuous
    'seed': 42
}

best_result = fmin(
    fn=objective_eval,
    space=search_space,
    algo=tpe.suggest, ## tpe is the algorithm that will be used to search the best set of hyper params!
    max_evals=50,
    trials=Trials() ## this will help in logging the metrics
)

# from math import exp
# exp(-3)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/opt/conda/envs/mlflow-exp/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [16:46:51] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:7.96712                           
[1]	validation-rmse:7.17219                           
[2]	validation-rmse:6.63019                           
[3]	validation-rmse:6.26892                           
[4]	validation-rmse:6.03103                           
[5]	validation-rmse:5.87361                           
[6]	validation-rmse:5.77080                           
[7]	validation-rmse:5.69881                           
[8]	validation-rmse:5.64649                           
[9]	validation-rmse:5.61076                           
[10]	validation-rmse:5.58429                          
[11]	validation-rmse:5.56399                          
[12]	validation-rmse:5.54878                          
[13]	validation-rmse:5.53658                          
[14]	validation-rmse:5.52699                          
[15]	validation-rmse:5.52056                          
[16]	validation-rmse:5.51341                          
[17]	validation-rmse:5.50708                          
[18]	valid

/opt/conda/envs/mlflow-exp/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [16:47:24] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:7.52304                                                   
[1]	validation-rmse:6.58523                                                   
[2]	validation-rmse:6.05328                                                   
[3]	validation-rmse:5.76330                                                   
[4]	validation-rmse:5.60751                                                   
[5]	validation-rmse:5.51620                                                   
[6]	validation-rmse:5.46718                                                   
[7]	validation-rmse:5.43883                                                   
[8]	validation-rmse:5.42288                                                   
[9]	validation-rmse:5.41201                                                   
[10]	validation-rmse:5.40405                                                  
[11]	validation-rmse:5.39577                                                  
[12]	validation-rmse:5.39333                        

...

/opt/conda/envs/mlflow-exp/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [16:48:16] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.21885                                                    
[1]	validation-rmse:7.53362                                                    
[2]	validation-rmse:7.01226                                                    
[3]	validation-rmse:6.61966                                                    
[4]	validation-rmse:6.32885                                                    
[5]	validation-rmse:6.11362                                                    
[6]	validation-rmse:5.95512                                                    
[7]	validation-rmse:5.83843                                                    
[8]	validation-rmse:5.75331                                                    
[9]	validation-rmse:5.68980                                                    
[10]	validation-rmse:5.64352                                                   
[11]	validation-rmse:5.60813                                                   
[12]	validation-rmse:5.58185            

/opt/conda/envs/mlflow-exp/lib/python3.10/site-packages/xgboost/callback.py:386: UserWarning: [16:49:01] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.67908                                                    
[1]	validation-rmse:8.29787                                                    
[2]	validation-rmse:7.95872                                                    
[3]	validation-rmse:7.65785                                                    
[4]	validation-rmse:7.39162                                                    
[5]	validation-rmse:7.15672                                                    
[6]	validation-rmse:6.94992                                                    
[7]	validation-rmse:6.76880                                                    
[8]	validation-rmse:6.61029                                                    
[9]	validation-rmse:6.47173                                                    
[10]	validation-rmse:6.35060                                                   
[11]	validation-rmse:6.24532                                                   
[12]	validation-rmse:6.15345            

KeyboardInterrupt: 

In [ ]:
arr = [1,2,3]
def rec(idx, arr):
    print(arr[idx])
    if idx==len(arr)-1:
        return
    # print(arr[idx])
    rec(idx+1,arr)

rec(0,arr)



1
2
